In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import data_cleaning

# Summary

This is an analysis originated from a bounty from [DiamondDAO](https://docs.diamonddao.xyz/welcome/values) that aims to analyze the data coming form ETH Denver attendees and their pathway in crypto after the event.

**Main objective:** 
- To determine the profile of inviduals attending the event.
- To understand the impacts of the event in the ecosystem. 

# Data loading

In [2]:
#POAP data
poap = data_cleaning.get_cleaned_poap_data()

#DAOHaus data
dh_members, dh_votes, dh_proposals = data_cleaning.get_daohaus_cleaned_data()

#Snapshot data
snap_proposals, snap_votes = data_cleaning.get_snapshot_cleaned_data()

#Token balance data

spork = data_cleaning.get_token_holder_cleaned_data()

In [3]:
spork.head()

,holder_address,balance,normalized_balance
0,0x70129EA2f8c3e4CA8C45621A5eC73a5A93a466D3,1.902079e+21,1902.07871
1,0x62051BfD3A4f7039a849142e6E5ea172cBdA5949,5.803448e+21,5803.44828
2,0x0c81D7d494faCFBa3eFE7A3030755Fc2249EA983,4.676353e+21,4676.35311
3,0x86096686BBC522491FFD5ad694929D6fc0DA952E,1.902079e+21,1902.07871
4,0x0e9B85B830f694f8A792bA721DfFe06c7E256588,2.000000e+21,2000.00000


# Analysis

Things to check:

- What is the proportion of ETH Denver POAP holders vs SPORK DAO holders.. the difference between these two populations
- What is the impact of the event in first time attendees? Is this introducing people to the ecosystem? what about continous participants
- What is the % of people that sticks to the event overtime


In [ ]:
# Check all ETH Denver POAP recipients
## What other events these people go? Any identifiable category or pattern here? 
## What is the rentention rate for ETH Denver? 
## Does ETH Denver lead to other events in a strong way?

# Cross reference POAP holders and SPORK holders
## make analysis on how much of the the POAP issuers have token of the opposite - which parameters identify the most people?


# Isolate all possible ETH Denver participants and start looking at DAO cross reference like:
## how many DAOs can the person be identified at (DAOHaus)
## how many proposals or votes this people have? once you have proposals or votes this would add you as member of the DAO
## explore the concept of stronger membership and weak membership attached to how much people participate 
## explore level of activity
### how many votes
### how many different DAOs
### total number of interactions vs count of specific reactions: vote, proposals, etc
### what is the SPORK holders share in the votes in snapshot - what is their wight on these things? vs only POAP vs general public

